In [1]:
from enum import StrEnum

class SevenTalents(StrEnum):
    """Level 7 talent options."""

    BURNINGBLADE = "Burning Blade"
    CRUSHINGBLOWS = "Crushing Blows"
    PHANTOMPAIN = "Phantom Pain"
    NONE = "None"

class OneTalents(StrEnum):
    """Level 1 talent options (That Matter)."""

    WAYOFILLUSION = "Way of Illusion"
    WAYOFTHEBLADE = "Way of the Blade"
    NONE = "None"

In [27]:
from dataclasses import dataclass

@dataclass
class Counters:
    """Mutable class to store counter information."""
    aa_damage: float
    crit_damage: float
    aa_speed: float
    attack_cadence: float
    remaining_w_cd: int = 0 # remaining W cooldown, defaults to 10 seconds
    crit_counter: int = 0 # counter for crits - we crit every 4th attack by default and every 3rd with WOTB
    cb_counter: int = 0 # Stacks of crushing blows. We start at 1 because we began with a preloaded crit.
    wotb_stacks: int = 0 # Stacks of Way of the Blade


In [24]:
def apply_crit(summed_damage: float, precb_aa_damage: float, counters: Counters, one_talent: OneTalents, seven_talent: SevenTalents, w_triggered: bool = False) -> float:
    """Applies a critical strike and returns the result."""
    # Critical Strike
    critModifier = 1.5
    base_w_cd = 10

    # Crushing Blows
    cbModifier = 0.1

    # Burning Blade
    bb_damage = 0.5 * counters.aa_damage

    if w_triggered:
        counters.remaining_w_cd = base_w_cd
    
    # Account for crushing blows' damage increase. CB applies to the auto attack that triggers it.
    if seven_talent == SevenTalents.CRUSHINGBLOWS:
        if counters.cb_counter < 3:
            counters.cb_counter += 1
            counters.aa_damage = precb_aa_damage * (1 + (cbModifier * counters.cb_counter))
            counters.crit_damage = precb_aa_damage * (critModifier + (cbModifier * counters.cb_counter))

    counters.crit_counter = 0
    summed_damage += counters.crit_damage + (counters.crit_damage * 0.05 * counters.wotb_stacks)
    if one_talent == OneTalents.WAYOFTHEBLADE and counters.wotb_stacks < 3:
        counters.wotb_stacks += 1
    if seven_talent == SevenTalents.BURNINGBLADE:
        summed_damage += bb_damage
    if w_triggered:
        print(summed_damage, "CRIT", "-W")
    else:
        print(summed_damage, "CRIT")
    return summed_damage

In [23]:
def raise_for_invalid_inputs(one_talent: OneTalents, seven_talent: SevenTalents, level: int, total_time: int) -> None:
    """Raises for invalid inputs."""
    if seven_talent == SevenTalents.PHANTOMPAIN:
        raise ValueError("cmon, really?")
    if total_time < 0:
        raise ValueError("total_time must be a non-negative number")
    if level < 0 or level > 30:
        raise ValueError("level must be a valid hots level between 0 and 30.")
    if (one_talent != OneTalents.NONE) and level == 0:
        raise ValueError("Cannot have Way of Illusion or Way of the Blade active at level 0.")
    if seven_talent != SevenTalents.NONE and level < 7:
        raise ValueError("Cannot have a level 7 talent without being at least level 7.")  

In [28]:
def damage_calc(level: int, total_time: int, one_talent: OneTalents=OneTalents.NONE, seven_talent: SevenTalents = SevenTalents.NONE) -> tuple[list[float], list[float]]:
    """Calculates a list of times and damage values for a given length of time"""

    # check for invalid inputs
    raise_for_invalid_inputs(one_talent, seven_talent, level, total_time)

    # # Globals
    aa_damage = 102.0 # level 0
    aa_speed = 1.67 # attacks per second
    attack_cadence = 1/aa_speed # seconds per attack
    aa_reset_time = 3/16 # seconds, approximately 3 game ticks.
    crit_modifier = 1.5

    # Way of Illusion
    woi_damage_increase = 40 # full stacks
    
    # Setup our Looping Variables
    passed_time = 0 # total time passed in the simulation
    summed_damage = 0 # total damage dealt
    crit_threshold = 2 if one_talent == OneTalents.WAYOFTHEBLADE else 3 # crit every 3rd attack with WOTB
    times = [] # Array of timestamps
    damages = [] # assume pre-loaded crit, array of damages

    # Initialize our counters
    # Recalculate our AA and Crit Damage based on talents and level
    counters = Counters(
        aa_damage = aa_damage * (1.04 ** level),
        crit_damage = aa_damage * (1.04 ** level) * 1.5,
        crit_counter=crit_threshold,
        aa_speed=aa_speed,
        attack_cadence=attack_cadence
    )
    if one_talent == OneTalents.WAYOFILLUSION:
        counters.aa_damage += woi_damage_increase
        counters.crit_damage = counters.aa_damage * crit_modifier

    # We start with a pre-loaded crit, so CB will activate immediately.
    # For CB to not calculate badly, we preserve the original damage number.
    precb_aa_damage = counters.aa_damage

    # Main Loop
    while passed_time < total_time:

        # Time passes
        counters.remaining_w_cd -= counters.attack_cadence
        times.append(passed_time)

        # Apply our damage - either a crit or an AA
        if counters.crit_counter == crit_threshold:
           summed_damage = apply_crit(summed_damage, precb_aa_damage, counters, one_talent, seven_talent)
        else:
            counters.crit_counter += 1
            summed_damage += counters.aa_damage + (counters.aa_damage * 0.05 * counters.wotb_stacks)
            print(summed_damage, "AA")
        damages.append(summed_damage)

        if seven_talent == SevenTalents.CRUSHINGBLOWS:
            counters.remaining_w_cd -= 2

        # Check if we can use W before we run out of time
        if (passed_time + aa_reset_time) > total_time:
            break
        
        # Apply W if we can and AA reset attack
        if counters.remaining_w_cd <= 0 and counters.crit_counter != crit_threshold:
            summed_damage = apply_crit(summed_damage, precb_aa_damage, counters, one_talent, seven_talent, True)
            damages.append(summed_damage)

            passed_time += aa_reset_time
            times.append(passed_time)

            if seven_talent == SevenTalents.CRUSHINGBLOWS:
                counters.remaining_w_cd -= 2

        # Increment time
        passed_time += counters.attack_cadence

    return times, damages


In [29]:
time_series, damage_series = damage_calc(7, 10, one_talent=OneTalents.WAYOFILLUSION, seven_talent=SevenTalents.CRUSHINGBLOWS)

278.76006637128916 CRIT
574.9426368907839 CRIT -W
784.0126866692507 AA
993.0827364477175 AA
1202.1527862261844 AA
1515.7578608938848 CRIT
1829.362935561585 CRIT -W
2055.8554894882573 AA
2282.3480434149296 AA
2508.840597341602 AA
2822.445672009302 CRIT
3136.0507466770023 CRIT -W
3362.5433006036747 AA
3589.035854530347 AA
3815.5284084570194 AA
4129.1334831247195 CRIT
4442.73855779242 CRIT -W
4669.231111719092 AA
4895.723665645764 AA
5122.216219572437 AA


In [22]:
time_series

[0,
 0.1875,
 0.7863023952095809,
 1.3851047904191618,
 1.9839071856287427,
 2.5827095808383236,
 2.7702095808383236,
 3.3690119760479043,
 3.967814371257485,
 4.566616766467066,
 5.165419161676646,
 5.352919161676646,
 5.951721556886227,
 6.550523952095808,
 7.149326347305388,
 7.748128742514969,
 7.935628742514969,
 8.53443113772455,
 9.133233532934131,
 9.732035928143713]

In [25]:
import matplotlib.pyplot as plt